In [2]:
print("hello2")

hello2


In [3]:
import csv
import subprocess
import getpass
import os
import gzip
from os import listdir
from os.path import isfile, join
from SciServer import Authentication

myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = getpass.getpass('Password for ' + myUserName + ': ')
userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = input("Please enter the name of your SAFE Desktop project folder : ") 
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname
devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

Password for mkim239:  ··········
Please enter the name of your SAFE Desktop project folder :  LCICM


In [33]:
# extracting patient_id of patients that have ECG recordings
# UPDATE: changing to patients that have 12-lead ECG recordings
patient_groups = [ f.path for f in os.scandir('SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/') if f.is_dir ]
print(len(patient_groups))

1000


In [4]:
pip install wfdb pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [7]:
import wfdb
import pandas as pd
import pyarrow
print("import successful")

import successful


In [13]:
def make_parquet(record_name, parquet_name):
    # ex record name: 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1000/p10000032/s40689238/40689238'
    record = wfdb.rdrecord(record_name)
    ecg_data = record.p_signal
    lead_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    ecg_df = pd.DataFrame(data=ecg_data, columns=lead_names)    
    ecg_df.to_parquet(parquet_name, index=False, engine='pyarrow')

,I,II,III,aVR,aVL,aVF,V1,V2,V3,V4,V5,V6
0,-0.155,0.01,0.150,0.070,0.075,-0.140,0.040,0.040,0.010,0.040,0.080,0.020
1,-0.170,0.01,0.165,0.075,0.085,-0.160,0.055,0.055,0.020,0.030,0.080,0.020
2,-0.170,0.00,0.155,0.080,0.075,-0.155,0.060,0.055,0.030,0.025,0.080,0.010
3,-0.170,0.00,0.155,0.080,0.075,-0.155,0.055,0.055,0.030,0.020,0.080,0.010
4,-0.170,0.00,0.155,0.080,0.075,-0.155,0.050,0.055,0.030,0.020,0.080,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-0.050,0.01,0.045,0.015,0.025,-0.040,-0.005,0.010,-0.030,-0.015,-0.010,-0.190
4996,-0.045,0.01,0.040,0.015,0.020,-0.035,-0.010,0.000,-0.040,-0.020,-0.015,-0.195
4997,-0.045,0.01,0.040,0.015,0.020,-0.035,-0.020,-0.015,-0.045,-0.020,-0.020,-0.200
4998,-0.055,0.00,0.040,0.025,0.015,-0.040,-0.010,-0.015,-0.045,-0.020,-0.020,-0.200


In [30]:
def get_time_ecg(file_path):
    with open(file_path, 'r') as file:
        # get time
        lines = file.readlines()
        first_line = lines[0].split()
        time = first_line[-2:]
        time = '_'.join(time)
        
        # get patient id
        last_line = lines[-1].split()
        patient_id = last_line[-1]
        full_name = patient_id + "_" + time + ".parquet"
        
        # TODO:
        # ADD FILE PATH FOR WHERE TO STORE PARQUET HERE:
        return full_name
    

In [31]:
print(get_time_ecg('SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1000/p10000032/s40689238/40689238.hea'))

10000032_08:44:00_23/07/2180.parquet


In [34]:
# basic ECG count & extracting the full list of patient IDs
ecg_ids = []
for patient_group in patient_groups:
    for f in os.scandir(patient_group):
        if f.is_dir():
            ecg_ids.append(int(f.name[1:]))
print(len(ecg_ids))

160720


In [ ]:
# basic ECG count & extracting the full list of patient IDs
ecg_ids = []
# every patient group
for patient_group in patient_groups:
    # every patient
    for f in os.scandir(patient_group):
        if f.is_dir():
            # every study id
            for f2 in os.scandir(f):
                if f2.is_dir():
                    # 0 => .dat file 
                    # 1 => .hea file
                    files = os.listdir(f2)
                    dat = os.path.join(f2, files[0])
                    hea = os.path.join(f2, files[1])
                    parquet_name = get_time_ecg(hea)
                    make_parquet(dat, parquet_name)
    break